# Fisheries

## VGG + BN + Multi OP

In [1]:
from theano.sandbox import cuda
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


ImportError: No module named utils

In [ ]:
#path = "data/fish/sample/"
path = "/home/ubuntu/data/fisheries/"
batch_size=64
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(path+'test', batch_size=batch_size*2, shuffle=False)

(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

In [2]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(8)

### Pre compute conv features

In [3]:
#Split Conv layers and create conv model
conv_layers,fc_layers = split_at_last(model, Convolution2D)
conv_model = Sequential(conv_layers)

In [ ]:
conv_feat = conv_model.predict(trn)
conv_val_feat = conv_model.predict(val)
conv_test_feat = conv_model.predict(test

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_feat.dat', conv_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [14]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [10]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

In [19]:
conv_feat.shape

(3277, 512, 14, 14)

### Stick a few BN and Dense layers and have multiple output layers

In [54]:
#MaxP+BN+ DO+Flat+Dense+ (BN+DO+Dense)+(BN+DO+Dense(box)+Dense(fish))
p=0.5
inp = Input(shape=conv_layers[-1].output_shape[1:])
x = MaxPooling2D()(inp)
x = BatchNormalization(axis=1)(x)
x = Dropout(p)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(p)(x)
x_bb = Dense(4, name='bb')(x)
x_fish = Dense(8, activation='softmax')(x)

In [55]:
model = Model([inp], [x_bb, x_fish])
model.compile(Adam(lr=0.001), loss=['mse', 'categorical_crossentropy'], metrics=['accuracy'],
             loss_weights=[.001, 1.])

### Load hand labelled bounding box training data and process them

In [35]:
import ujson as json
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']
bb_json = {}
for c in anno_classes:
    j = json.load(open('{}anno/{}_labels.json'.format(path, c), 'r'))
    #Create a map with image name as key and height/width,x,y as values 
    #and ordered by size of the box
    #Note: Some filenames are absolute, so clean them up
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

In [40]:
bb_json['img_05166.jpg']

{u'class': u'rect',
 u'height': 109.98000000000036,
 u'width': 201.6300000000006,
 u'x': 208.6800000000006,
 u'y': 359.55000000000103}

#### Add empty boxes for images which do not have fishes

In [ ]:
# Get the actual file name
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

In [41]:
#Create an indexed list of names?
file2idx = {o:i for i,o in enumerate(raw_filenames)}
val_file2idx = {o:i for i,o in enumerate(raw_val_filenames)}

In [45]:
empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}
for f in raw_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox
for f in raw_val_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox

#### Normalize it to 224x224 image size

In [46]:
bb_params = ['height', 'width', 'x', 'y']
def convert_bb(bb, size):
    bb = [bb[p] for p in bb_params]
    conv_x = (224. / size[0])
    conv_y = (224. / size[1])
    bb[0] = bb[0]*conv_y
    bb[1] = bb[1]*conv_x
    bb[2] = max(bb[2]*conv_x, 0)
    bb[3] = max(bb[3]*conv_y, 0)
    return bb

In [48]:
sizes = [PIL.Image.open(path+'train/'+f).size for f in filenames]

In [47]:
trn_bbox = np.stack([convert_bb(bb_json[f], s) for f,s in zip(raw_filenames, sizes)], 
                   ).astype(np.float32)
val_bbox = np.stack([convert_bb(bb_json[f], s) 
                   for f,s in zip(raw_val_filenames, raw_val_sizes)]).astype(np.float32)

In [56]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/3
3277/3277 [==============================] - 2s - loss: 6.3012 - bb_loss: 5040.5669 - dense_13_loss: 1.2606 - bb_acc: 0.4275 - dense_13_acc: 0.6375 - val_loss: 5.5811 - val_bb_loss: 4634.0021 - val_dense_13_loss: 0.9471 - val_bb_acc: 0.5600 - val_dense_13_acc: 0.8140
Epoch 2/3
3277/3277 [==============================] - 1s - loss: 5.1448 - bb_loss: 4781.7002 - dense_13_loss: 0.3631 - bb_acc: 0.5288 - dense_13_acc: 0.9008 - val_loss: 4.6197 - val_bb_loss: 4282.2805 - val_dense_13_loss: 0.3374 - val_bb_acc: 0.5420 - val_dense_13_acc: 0.9380
Epoch 3/3
3277/3277 [==============================] - 1s - loss: 4.5522 - bb_loss: 4376.1161 - dense_13_loss: 0.1761 - bb_acc: 0.5590 - dense_13_acc: 0.9445 - val_loss: 4.0888 - val_bb_loss: 3853.0484 - val_dense_13_loss: 0.2358 - val_bb_acc: 0.5640 - val_dense_13_acc: 0.9540


In [57]:
model.optimizer.lr = 1e-5

In [58]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/3
3277/3277 [==============================] - 2s - loss: 3.9616 - bb_loss: 3836.2062 - dense_13_loss: 0.1254 - bb_acc: 0.5816 - dense_13_acc: 0.9628 - val_loss: 3.6075 - val_bb_loss: 3367.5271 - val_dense_13_loss: 0.2399 - val_bb_acc: 0.6180 - val_dense_13_acc: 0.9620
Epoch 2/3
3277/3277 [==============================] - 1s - loss: 3.2960 - bb_loss: 3202.8387 - dense_13_loss: 0.0932 - bb_acc: 0.6079 - dense_13_acc: 0.9735 - val_loss: 3.1080 - val_bb_loss: 2849.2987 - val_dense_13_loss: 0.2587 - val_bb_acc: 0.6280 - val_dense_13_acc: 0.9520
Epoch 3/3
3277/3277 [==============================] - 1s - loss: 2.5242 - bb_loss: 2466.3798 - dense_13_loss: 0.0578 - bb_acc: 0.6430 - dense_13_acc: 0.9835 - val_loss: 2.2514 - val_bb_loss: 2024.4412 - val_dense_13_loss: 0.2269 - val_bb_acc: 0.6620 - val_dense_13_acc: 0.9640


In [59]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/3
3277/3277 [==============================] - 2s - loss: 1.7986 - bb_loss: 1742.1811 - dense_13_loss: 0.0565 - bb_acc: 0.6771 - dense_13_acc: 0.9847 - val_loss: 1.6382 - val_bb_loss: 1383.1369 - val_dense_13_loss: 0.2551 - val_bb_acc: 0.7020 - val_dense_13_acc: 0.9560
Epoch 2/3
3277/3277 [==============================] - 1s - loss: 1.1535 - bb_loss: 1100.6106 - dense_13_loss: 0.0529 - bb_acc: 0.7202 - dense_13_acc: 0.9854 - val_loss: 1.1818 - val_bb_loss: 923.7057 - val_dense_13_loss: 0.2581 - val_bb_acc: 0.7420 - val_dense_13_acc: 0.9600
Epoch 3/3
3277/3277 [==============================] - 1s - loss: 0.7354 - bb_loss: 681.8768 - dense_13_loss: 0.0535 - bb_acc: 0.7711 - dense_13_acc: 0.9857 - val_loss: 0.8161 - val_bb_loss: 578.8653 - val_dense_13_loss: 0.2373 - val_bb_acc: 0.7960 - val_dense_13_acc: 0.9620


In [119]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/3
3277/3277 [==============================] - 2s - loss: 0.3070 - bb_loss: 278.1977 - dense_13_loss: 0.0288 - bb_acc: 0.7968 - dense_13_acc: 0.9915 - val_loss: 0.6512 - val_bb_loss: 365.4737 - val_dense_13_loss: 0.2858 - val_bb_acc: 0.8200 - val_dense_13_acc: 0.9600
Epoch 2/3
3277/3277 [==============================] - 1s - loss: 0.2889 - bb_loss: 262.5483 - dense_13_loss: 0.0264 - bb_acc: 0.8007 - dense_13_acc: 0.9924 - val_loss: 0.6311 - val_bb_loss: 358.0238 - val_dense_13_loss: 0.2731 - val_bb_acc: 0.8300 - val_dense_13_acc: 0.9580
Epoch 3/3
3277/3277 [==============================] - 1s - loss: 0.2813 - bb_loss: 255.7603 - dense_13_loss: 0.0255 - bb_acc: 0.8050 - dense_13_acc: 0.9921 - val_loss: 0.6233 - val_bb_loss: 360.0164 - val_dense_13_loss: 0.2632 - val_bb_acc: 0.8180 - val_dense_13_acc: 0.9580


# Save weights

In [ ]:
model.save_weights(path+'results/multi_output.h5')

# Create submission

# Save results on training and validation sets

In [68]:
results = model.evaluate(conv_val_feat, [val_bbox, val_labels])

500/500 [==============================] - 0s     


In [81]:
import simplejson
with open(path+'results/multi_op.results', 'wb') as fp:
    simplejson.dump(results, fp)

## Generate predictions on test set and generate submission

In [82]:
preds = model.predict(conv_test_feat, batch_size=batch_size)

In [87]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)

In [88]:
preds = preds[1]

In [94]:
subm = do_clip(preds,0.90)
subm_name = path+'results/multi_op.gz'    

In [90]:
# classes = sorted(batches.class_indices, key=batches.class_indices.get)
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [91]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.266700,0.647696,0.014286,0.014286,0.014286,0.015809,0.014286,0.059853
1,img_06893.jpg,0.900000,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286
2,img_02082.jpg,0.233902,0.014286,0.014286,0.014286,0.759288,0.014286,0.014286,0.014286
3,img_06261.jpg,0.900000,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286
4,img_03628.jpg,0.900000,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286


In [95]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [96]:
FileLink(subm_name)

/home/ubuntu/data/fisheries/results/multi_op.gz

# Pseudo labelling

In [136]:
gen = image.ImageDataGenerator()

In [145]:
test_batches = gen.flow(conv_feat, trn_labels, batch_size=16)

In [146]:
val_batches = gen.flow(conv_val_feat, val_labels, batch_size=4)

In [147]:
batches = gen.flow(conv_feat, trn_labels, batch_size=44)

In [115]:
batches.y

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [148]:
mi = MixIterator([batches, test_batches, val_batches])

In [149]:
model.fit_generator(mi, mi.N, nb_epoch=8, validation_data=(conv_val_feat, val_labels))

Exception: The model expects 2 input arrays, but only received one array. Found: array with shape (500, 8)